In [1]:
import torch as tt

In [2]:
tt.cuda.is_available()

True

In [4]:
!pip install transformers

Using legacy 'setup.py install' for sacremoses, since package 'wheel' is not installed.
    Running setup.py install for sacremoses: started
    Running setup.py install for sacremoses: finished with status 'done'


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


In [3]:
from transformers import AutoTokenizer, AutoModel
from math import ceil, floor
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

import pandas as pd
import numpy as np
import torch as tt
import os
from torch import nn

In [4]:
tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-en")
model = AutoModel.from_pretrained("nghuyong/ernie-2.0-en")
device = tt.device('cuda')

In [5]:
class ErnieEmbedder:
  def __init__(self, model, device):
    self.embedder = model
    self.device = device
  
  def embed_src(self, src):
    embed_src = self.embedder(src)['last_hidden_state'].detach()
    embed_src = embed_src.to(self.device)
    embed_src = embed_src.permute(1,0,2)
    return embed_src
  
  def embed_tgt(self, tgt):
    embed_tgt = self.embedder(tgt)['last_hidden_state'].detach()
    embed_tgt = embed_tgt.to(self.device)
    embed_tgt = embed_tgt.permute(1,0,2)
    ## set start token vector to zero - for fair benchmarking
    embed_tgt[0] = tt.zeros((embed_tgt.size(1), embed_tgt.size(2)), dtype=tt.float32)
    return embed_tgt

embedder = ErnieEmbedder(model, device)

class MyModel(nn.Module):
   def encode(self, batch):
    with tt.no_grad():
      embed_batch = self.embedder.embed_src(batch)
    return self.encoder(embed_batch)

   def decode(self, src, tgt):
    with tt.no_grad():
      embed_tgt = self.embedder.embed_tgt(tgt)
    embed_tgt = embed_tgt[:-1:]
    decoded = self.decoder(embed_tgt, src)
    return self.logits(decoded)
  
   def decode_test(self, src, tgt):
    with tt.no_grad():
      embed_tgt = self.embedder.embed_tgt(tgt)
    #print(src.size(), embed_tgt.size())
    decoded = self.decoder(embed_tgt, src)
    return self.logits(decoded)
  
   def forward(self, src, tgt):
    inp = self.encode(src)
    outp = self.decode(inp, tgt)
    return outp
    
   def forward_test(self, src, max_len, method='argmax',k=5):
    if method == 'argmax' or max_len<3:
      start_vector = tt.Tensor([[self.sos_id] for i in range(src.size(0))]).long()
      inp = self.encode(src)
      for i in range(max_len-1):
        outp = self.decode_test(inp, start_vector)
        new_prediction = outp.argmax(2)[-1::].cpu().permute(1,0)
        start_vector = tt.cat((start_vector, new_prediction), dim=1)
      return outp
    elif method == 'beam_search':
      start_vector = tt.Tensor([[self.sos_id] for i in range(src.size(0))]).long()
      inp = self.encode(src)
      outp = self.decode_test(inp, start_vector)
      new_predictions = [tensor for tensor in tt.topk(outp,k,dim=2).cpu().permute(2,1,0)]
      start_vectors = [tt.cat((start_vector, new_prediction), dim=1) for new_prediction in new_predictions]
      for i in range(max_len-2):
        new_start_vectors = []
        for start_vector in start_vectors:
          outp = self.decode_test(inp, start_vector)
          new_prediction = outp.argmax(2)[-1::].cpu().permute(1,0)
          start_vector = tt.cat((start_vector, new_prediction), dim=1)
          new_start_vectors.append(start_vector)
        start_vectors = new_start_vectors
  
   def _init_params(self, device, embed_dim, sos_id):
    self.embed_dim = embed_dim
    self.device = device
    self.sos_id = sos_id
    self.to(self.device)

class MyLinearModel(MyModel):
  def __init__(self, n_vocab: int, n_hidden_layers: int, hidden_dims: list,
               embedder_model=embedder, embed_dim=768,
               device=tt.device('cuda'), sos_id=101):
    layers = []
    prev_dim = embed_dim
    for i in range(n_hidden_layers):
      layers.append(nn.Linear(prev_dim, hidden_dims[i]))
      prev_dim = hidden_dims[i]
    self.linear_layers = nn.Sequential(layers)
    self.logits = nn.Linear(prev_dim, n_vocab)
    self._init_params(device, embed_dim, sos_id)

class MyRNN(MyModel):
  def __init__(self, n_vocab: int, n_encoder_layers=1, n_decoder_layers=1,
               embedder_model=embedder, embed_dim=768,
               device=tt.device('cuda'), sos_id=101):
    pass
    
class MyTransformer(MyModel):
  def __init__(self, n_vocab: int, n_encoder_layers=1, n_decoder_layers=6,
               embedder_model=embedder, embed_dim=768, n_encoder_head=8, n_decoder_head=8,
               device=tt.device('cuda'), sos_id=101):
    super().__init__()
    self.embedder = embedder
    encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=n_encoder_head)
    self.encoder = nn.TransformerEncoder(encoder_layer, n_encoder_layers)
    decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=n_decoder_head)
    self.decoder = nn.TransformerDecoder(decoder_layer, n_decoder_layers )
    self.logits = nn.Linear(embed_dim, n_vocab)
    self._init_params(device, embed_dim, sos_id)
      

In [6]:
os.listdir()

['.ipynb_checkpoints',
 'ErnieHumour.ipynb',
 'my_model',
 'NewsPunchlinesLateNight.csv']

In [7]:
df = pd.read_csv("NewsPunchlinesLateNight.csv", index_col='Unnamed: 0')

In [8]:
df.head()

,Newsline,Punchline
0,"According to a new report, young Republican st...","According to some Republican staffers, they ar..."
1,President Trump is headed home after his trip ...,"And I saw that at one point, 2,000 protesters ..."
2,Scientists have developed a robot that can con...,"When the scientists unveiled the robot, it scr..."
3,"According to a new poll, one third of American...",The other two thirds have cats.
4,"My policy is, you should treat selfies like yo...","Try not to do it alone, definitely don't do it..."


In [9]:
df.shape

(17545, 2)

In [10]:
class MyBatch:
  def __init__(self, df):
    for col_name in df:
      setattr(self, col_name, df[col_name].values)

class MyBatchIterator:
  def __init__(self, df, batch_size):
    self.df = df
    self.batch_size = batch_size
  
  def __iter__(self):
    self.start = 0
    return self
  
  def __next__(self):
    if self.start >= self.df.shape[0]:
      raise StopIteration
    batch = self.df.iloc[self.start:self.start+self.batch_size]
    batch = MyBatch(batch)
    self.start += self.batch_size
    return batch
  
  def __len__(self):
    return ceil(self.df.shape[0]/self.batch_size)

In [11]:
def tokenize_batch(batch, tokenizer=tokenizer, max_length=32):
  X = tokenizer(batch.Newsline.tolist(), return_tensors='pt', padding=True, truncation=True, max_length=max_length)['input_ids']
  y = tokenizer(batch.Punchline.tolist(), return_tensors='pt', padding=True, truncation=True, max_length=max_length)['input_ids']
  return X, y

In [12]:
def embed_tokens(token_tensor, model=model):
  outp = model(token_tensor)[0].detach()
  return outp

In [13]:
batch_size = 64

train_df, test_df = train_test_split(df, test_size=0.2)
train_iter = MyBatchIterator(train_df, batch_size)
test_iter = MyBatchIterator(test_df, batch_size)

In [14]:
def fake_train(train_iter, n_epochs):
  for epoch in range(n_epochs):
    for batch in tqdm_notebook(train_iter, total=len(train_iter)):
      X, y = tokenize_batch(batch)
      X = embed_tokens(X)
      y = embed_tokens(y)

In [24]:
def train_epoch(train_iter, model, criterion, optimizer, device, epoch):
  model.train()
  batch_iter = tqdm_notebook(enumerate(train_iter), total=len(train_iter),
                             desc='Training epoch %d' % (epoch + 1), leave=True)
  running_loss = 0
  for i, batch in batch_iter:
    optimizer.zero_grad()
    src, tgt  = tokenize_batch(batch)
    pred = model(src, tgt)
    ## From (seq_len, batch_size, vocab_len) to (seq_len * batch_size, vocab_len)
    pred = pred.view(-1,pred.size(2))
    ## From (batch_size, seq_len) to (seq_len * batch_size)
    ground_truth = tgt.permute(1,0)[1::].reshape(-1).to(device)
    loss = criterion(pred, ground_truth)
    loss.backward()
    optimizer.step()

    curr_loss = loss.data.cpu().detach().item()
    loss_smoothing = i / (i+1)
    running_loss = loss_smoothing * running_loss + (1 - loss_smoothing) * curr_loss
    batch_iter.set_postfix(loss='%.4f' % running_loss)


def test_epoch(test_iter, model, criterion, device):
  model.eval()
  batch_iter = tqdm_notebook(enumerate(test_iter), total=len(test_iter),
                             desc='Testing', leave=True)
  test_loss = []
  with tt.no_grad():
    for i, batch in batch_iter:
      src, tgt = tokenize_batch(batch)
      max_len = tgt.size(1)
      pred = model.forward_test(src, max_len)
      ## print(pred.size(), tgt.size())
      ## From (seq_len, batch_size, vocab_len) to (seq_len * batch_size, vocab_len)
      pred = pred.view(-1,pred.size(2))
      ## From (batch_size, seq_len) to (seq_len * batch_size)
      ground_truth = tgt.permute(1,0)[1:].reshape(-1).to(device)
      loss = criterion(pred, ground_truth).data.cpu().detach().item()
      test_loss.append(loss)
  return np.mean(test_loss)
    
def train(model, n_epochs, train_iter, test_iter, criterion, optimizer, device, scheduler=None):
  prev_test_loss = float('inf')
  for epoch in range(n_epochs):
    train_epoch(train_iter, model, criterion, optimizer, device, epoch)
    test_loss = test_epoch(test_iter, model, criterion, device)
    if type(scheduler) == tt.optim.lr_scheduler.ReduceLROnPlateau:
      scheduler.step(test_loss)
    elif scheduler:
      scheduler.step()
    print('validation loss=%.4f' % test_loss)
    if test_loss < prev_test_loss:
      print("New record! - Saving model")
      tt.save(model.state_dict(), 'my_model')
      prev_test_loss = test_loss

In [16]:
tokenizer.vocab_size

30522

In [17]:
device=tt.device('cuda')

In [18]:
my_model = MyTransformer(n_vocab=tokenizer.vocab_size, device=device)

In [19]:
optimizer = tt.optim.Adam(my_model.parameters())

In [20]:
tokenizer.vocab[tokenizer.pad_token], tokenizer.vocab[tokenizer.cls_token], tokenizer.vocab[tokenizer.sep_token]

(0, 101, 102)

In [21]:
criterion = nn.CrossEntropyLoss(ignore_index=0).to(device)

In [23]:
train(my_model, n_epochs=200, train_iter=train_iter, test_iter=test_iter,
      optimizer=optimizer, criterion=criterion, device=device,
     scheduler=tt.optim.lr_scheduler.ReduceLROnPlateau(optimizer))

c:\python37\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


c:\python37\lib\site-packages\ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



validation loss=6.4969
New record! - Saving model



validation loss=6.5369
New record! - Saving model



validation loss=6.5477
New record! - Saving model


...


validation loss=6.5124
New record! - Saving model



validation loss=6.5124
New record! - Saving model



validation loss=6.5124
New record! - Saving model



validation loss=6.5124
New record! - Saving model



validation loss=6.5124
New record! - Saving model


In [25]:
def process(model, strings, tokenizer=tokenizer, max_len=20):
    model_input = tokenizer(strings, return_tensors='pt', padding=True, truncation=True, max_length=128)['input_ids']
    model_output = model.forward_test(src=model_input, max_len=max_len)
    model_output = model_output.argmax(dim=2).transpose(1,0)
    print(type(model_output), model_output.shape)
    return [tokenizer.decode(i) for i in model_output]

In [26]:
strings = [
    "According to a new report, young Republican staffers in Washington, D.C., are having trouble meeting people on dating apps because they're being harassed for their political views."
]

In [27]:
process(my_model, strings)

<class 'torch.Tensor'> torch.Size([1, 19])


['[SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]']

Проверим, что токенайзер нормально работает в обе стороны:

In [29]:
b = next(iter(train_iter))

In [30]:
x, y = tokenize_batch(b)

In [31]:
tokenizer.decode(x[0])

'[CLS] cher is turning 65 years old. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [36]:
tokenizer.decode(x[0]).count('[PAD]')

9

In [32]:
tokenizer.decode(y[0])

'[CLS] they had a party and friends said that she was alert, she smiled, and even had a little cake. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [33]:
tokenizer.decode(x[1])

'[CLS] tim tebow will officially sign a contract with the eagles. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [34]:
tokenizer.decode(y[1])

"[CLS] it is pretty shocking, mainly because i didn't even know he played an instrument. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"